## **IMPORTING DATA**

In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sales_df = pd.read_csv("data/retail_store_sales.csv", index_col=0)
sales_df.head()

,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
Transaction ID,,,,,,,,,,
TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [3]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12575 entries, TXN_6867343 to TXN_2407494
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Customer ID       12575 non-null  object 
 1   Category          12575 non-null  object 
 2   Item              11362 non-null  object 
 3   Price Per Unit    11966 non-null  float64
 4   Quantity          11971 non-null  float64
 5   Total Spent       11971 non-null  float64
 6   Payment Method    12575 non-null  object 
 7   Location          12575 non-null  object 
 8   Transaction Date  12575 non-null  object 
 9   Discount Applied  8376 non-null   object 
dtypes: float64(3), object(7)
memory usage: 1.1+ MB


In [4]:
sales_df.shape

(12575, 10)

## **Data Cleaning**

In [5]:
# checking for duplicates
sales_df_duplicates = sales_df.duplicated().value_counts()
sales_df_duplicates

False    12575
dtype: int64

In [6]:
# checking for missing values
sales_df.isna().sum()

Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [7]:
# filling in values in the price per unit
sales_df["Price Per Unit"] = sales_df["Price Per Unit"].fillna(sales_df["Total Spent"]/sales_df["Quantity"])
sales_df.isna().sum()

Customer ID            0
Category               0
Item                1213
Price Per Unit         0
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [8]:
# filling in values in Quantity
sales_df["Quantity"] = sales_df["Quantity"].fillna(sales_df["Quantity"].median())
sales_df.isna().sum()

Customer ID            0
Category               0
Item                1213
Price Per Unit         0
Quantity               0
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [9]:
#Filling in values in Total spent
sales_df["Total Spent"] = sales_df["Total Spent"].fillna(sales_df["Price Per Unit"]*sales_df["Quantity"])
sales_df.isna().sum()

Customer ID            0
Category               0
Item                1213
Price Per Unit         0
Quantity               0
Total Spent            0
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [10]:
# filling in values in Item
# creating a dictionary with the items and prices
item_price = dict(zip(sales_df['Item'],sales_df['Price Per Unit']))

from collections import defaultdict

# create a dictionary that map in reverse
price_item = defaultdict(list)
for item, price in item_price.items():
    price_item[price].append(item)

# handling null values in the Item column using reverse map
sales_df['Item'] = sales_df.apply(
    lambda row : np.random.choice(price_item[row['Price Per Unit']])
    if pd.isnull(row['Item']) and row['Price Per Unit'] in price_item
    else row['Item'],
    axis=1
)

sales_df.isna().sum()

Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity               0
Total Spent            0
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [14]:
sales_df = sales_df.drop("Discount Applied", axis=1)
sales_df.isna().sum()

Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [20]:
# printing unique values in categorical columns
cat_cols = sales_df.select_dtypes(include=['object']).columns.to_list()
#excluding column
cat_cols = [col for col in cat_cols if col not in ['Transaction ID','Customer ID','Transaction Date']]

for col in cat_cols:
    if sales_df[col].nunique() > 10:
        print(f'{col}: {sales_df[col].nunique()}\n')
    else:
        print(f'{sales_df[col].value_counts()}\n')

Furniture                             1591
Electric household essentials         1591
Food                                  1588
Milk Products                         1584
Butchers                              1568
Beverages                             1567
Computers and electric accessories    1558
Patisserie                            1528
Name: Category, dtype: int64

Item: 201

Cash              4310
Digital Wallet    4144
Credit Card       4121
Name: Payment Method, dtype: int64

Online      6354
In-store    6221
Name: Location, dtype: int64



In [22]:
#Correcting the data type in the Transaction Date column
sales_df['Transaction Date'] = pd.to_datetime(sales_df['Transaction Date'])

In [23]:
sales_df.head()

,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date
Transaction ID,,,,,,,,,
TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08
TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23
TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05
TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07
TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02


In [28]:
sales_df.to_csv("Cleaned_sales_data.csv", index=False)